I CANT EXPLAIN HOW MUCH I HATE PREPROCESSING DATA 

In [1]:
import mne
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
#importing stuff

In [3]:
file_path = r'D:\coding\training\Epileptic_Seizure_Recognition.fif'
raw = mne.io.read_raw_fif(file_path, preload=True)
#load the data

Opening raw data file D:\coding\training\Epileptic_Seizure_Recognition.fif...
Isotrak not found
    Range : 0 ... 11499 =      0.000 ...    44.918 secs
Ready.
Reading 0 ... 11499  =      0.000 ...    44.918 secs...


C:\Users\YUVRAJ SINGH\AppData\Local\Temp\ipykernel_12328\1308944251.py:2: RuntimeWarning: This filename (D:\coding\training\Epileptic_Seizure_Recognition.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(file_path, preload=True)


In [4]:
raw.filter(1., 40., fir_design='firwin')
#filter

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 845 samples (3.301 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.0s


Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,179 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,256.00 Hz
Highpass,1.00 Hz
Lowpass,40.00 Hz


In [5]:
epoch_length = 1.0  # in seconds
sfreq = raw.info['sfreq']  # sampling frequency
n_samples = int(epoch_length * sfreq)

In [6]:
data = raw.get_data()

In [7]:
n_epochs = data.shape[1] // n_samples
X = np.array([data[:, i*n_samples:(i+1)*n_samples] for i in range(n_epochs)])

In [8]:
y = np.random.randint(0, 2, n_epochs)

In [9]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [10]:
print("Data shape:", X.shape)
print("Labels shape:", y.shape)

Data shape: (44, 179, 256)
Labels shape: (44,)


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
np.save('X_train.npy', X_train)
np.save('X_test.npy', X_test)
np.save('y_train.npy', y_train)
np.save('y_test.npy', y_test)

This is the actual part where i make the model

In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
#import evrythang

In [14]:
model = Sequential()

In [15]:
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

C:\Users\YUVRAJ SINGH\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [21]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

here comes the training part

In [22]:
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5357 - loss: 0.6906 - val_accuracy: 0.7143 - val_loss: 0.6650
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.5357 - loss: 0.6906 - val_accuracy: 0.7143 - val_loss: 0.6650
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.5357 - loss: 0.6906 - val_accuracy: 0.7143 - val_loss: 0.6650
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.5357 - loss: 0.6906 - val_accuracy: 0.7143 - val_loss: 0.6650
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.5357 - loss: 0.6906 - val_accuracy: 0.7143 - val_loss: 0.6650
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.5357 - loss: 0.6906 - val_accuracy: 0.7143 - val_loss: 0.6650
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.5357 - loss: 0.6906 - val_accuracy: 0.7143 - val_loss: 0.6650
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.5357 - loss: 0.6906 - val_accuracy: 0.7143 - val_loss: 0